In [1]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# 1. SETUP
PROJECT_ID = "altdatapulsedashboard" 
TICKER = "SBUX"                     # Starbucks
BRAND_NAME = "Starbucks"            # As it appears in Consumer Edge

In [3]:
# 2. GET ALTERNATIVE DATA (Consumer Edge)
print(f"Fetching Spend Data for {BRAND_NAME}...")
sql = f"""
SELECT 
    date,
    spend,
    transactions
FROM `alternative_data.clean_revenue_daily`
WHERE brand = '{BRAND_NAME}'
ORDER BY date
"""
df_alt = pd.read_gbq(sql, project_id=PROJECT_ID)
df_alt['date'] = pd.to_datetime(df_alt['date'])

Fetching Spend Data for Starbucks...


/var/folders/y2/yt_1qz7913jf55m81j1hs4340000gn/T/ipykernel_93820/1928093338.py:12: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_alt = pd.read_gbq(sql, project_id=PROJECT_ID)
/Users/aaron/opt/anaconda3/envs/pulse/lib/python3.12/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
# 3. GET MARKET DATA (Stock Price)
# We use Stock Price as a proxy for "Market Expectation/Performance" for now
print(f"Fetching Market Data for {TICKER}...")
df_market = yf.download(TICKER, start="2020-01-01", end="2025-12-31", progress=False)
df_market = df_market.reset_index()

# Fix for newer yfinance: flatten MultiIndex columns
if isinstance(df_market.columns, pd.MultiIndex):
    df_market.columns = [c[0] for c in df_market.columns]

df_market.columns = [c.lower() for c in df_market.columns]
print(f"Retrieved {len(df_market)} rows of market data")
df_market.head()

Fetching Market Data for SBUX...
Retrieved 1507 rows of market data


,date,close,high,low,open,volume
0,2020-01-02,78.221725,78.221725,77.083641,77.144922,6473800
1,2020-01-03,77.766479,78.002849,77.127396,77.617654,4874100
2,2020-01-06,77.153679,77.398811,76.575883,77.136173,5409800
3,2020-01-07,76.917297,77.066123,76.278214,77.004841,6514700
4,2020-01-08,77.810280,78.221744,76.847283,76.987358,7296300


In [5]:
# 4. DATA PROCESSING (The "Signal Construction")
# Daily data is noisy. We need a moving average to see the trend.
df_alt['rolling_spend_7d'] = df_alt['spend'].rolling(window=7).mean()
df_alt['rolling_spend_30d'] = df_alt['spend'].rolling(window=30).mean()

In [6]:
# 5. VISUALIZATION (The "Mosaic" Chart)
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot Alt Data (Spend)
fig.add_trace(
    go.Scatter(x=df_alt['date'], y=df_alt['rolling_spend_30d'], name="Consumer Edge Spend (30d Avg)", line=dict(color='blue')),
    secondary_y=False
)

In [7]:
# Check if df_alt has any data
print(f"df_alt shape: {df_alt.shape}")
print(f"df_alt head:\n{df_alt.head()}")


df_alt shape: (0, 5)
df_alt head:
Empty DataFrame
Columns: [date, spend, transactions, rolling_spend_7d, rolling_spend_30d]
Index: []


In [8]:
# Check what brands exist in the table
sql_brands = """
SELECT DISTINCT brand
FROM `alternative_data.clean_revenue_daily`
LIMIT 20
"""
brands = pd.read_gbq(sql_brands, project_id=PROJECT_ID)
print("Available brands:")
print(brands)


/var/folders/y2/yt_1qz7913jf55m81j1hs4340000gn/T/ipykernel_93820/4166477814.py:7: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



Available brands:
Empty DataFrame
Columns: [brand]
Index: []


In [9]:
# Check what tables exist in the alternative_data dataset
sql_tables = """
SELECT table_name 
FROM `alternative_data.INFORMATION_SCHEMA.TABLES`
"""
try:
    tables = pd.read_gbq(sql_tables, project_id=PROJECT_ID)
    print("Tables in alternative_data dataset:")
    print(tables)
except Exception as e:
    print(f"Error: {e}")


/var/folders/y2/yt_1qz7913jf55m81j1hs4340000gn/T/ipykernel_93820/857618329.py:7: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



Tables in alternative_data dataset:
                  table_name
0  mart_weekly_revenue_proxy
1    stg_consumer_edge_daily
2            stg_advan_daily
3    raw_consumer_edge_daily
4        clean_revenue_daily
5           raw_advan_weekly


In [10]:
# Check raw_consumer_edge_daily for data
sql_raw = """
SELECT *
FROM `alternative_data.raw_consumer_edge_daily`
LIMIT 5
"""
try:
    raw_data = pd.read_gbq(sql_raw, project_id=PROJECT_ID)
    print(f"raw_consumer_edge_daily shape: {raw_data.shape}")
    print(raw_data)
except Exception as e:
    print(f"Error: {e}")


/var/folders/y2/yt_1qz7913jf55m81j1hs4340000gn/T/ipykernel_93820/2932312690.py:8: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



raw_consumer_edge_daily shape: (5, 6)
   BRAND_ID      BRAND_NAME  SPEND_AMOUNT  TRANS_COUNT  TRANS_DATE     VERSION
0     10001  1-800 CONTACTS     188910.17      1092.47  2018-07-13  2025-10-06
1     10001  1-800 CONTACTS     257772.35      1271.16  2025-03-15  2025-10-06
2     10001  1-800 CONTACTS      47702.76       385.61  2018-12-14  2025-10-06
3     10001  1-800 CONTACTS     348877.51      1771.78  2023-04-03  2025-10-06
4     10001  1-800 CONTACTS     307595.86      1683.08  2025-06-27  2025-10-06


In [11]:
# Check if Starbucks exists in raw data
sql_sbux = """
SELECT DISTINCT BRAND_NAME
FROM `alternative_data.raw_consumer_edge_daily`
WHERE UPPER(BRAND_NAME) LIKE '%STARBUCK%'
"""
sbux = pd.read_gbq(sql_sbux, project_id=PROJECT_ID)
print("Starbucks in raw data:")
print(sbux)


/var/folders/y2/yt_1qz7913jf55m81j1hs4340000gn/T/ipykernel_93820/3364105969.py:7: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



Starbucks in raw data:
             BRAND_NAME
0  STARBUCKS (MERCHANT)
1        STARBUCKS CARD


In [12]:
# Query directly from raw data
sql_raw_sbux = """
SELECT 
    TRANS_DATE as date,
    SUM(SPEND_AMOUNT) as spend,
    SUM(TRANS_COUNT) as transactions
FROM `alternative_data.raw_consumer_edge_daily`
WHERE BRAND_NAME = 'STARBUCKS (MERCHANT)'
GROUP BY TRANS_DATE
ORDER BY TRANS_DATE
"""
df_alt = pd.read_gbq(sql_raw_sbux, project_id=PROJECT_ID)
df_alt['date'] = pd.to_datetime(df_alt['date'])
print(f"Retrieved {len(df_alt)} rows of alt data")
df_alt.head()


/var/folders/y2/yt_1qz7913jf55m81j1hs4340000gn/T/ipykernel_93820/1222284369.py:12: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



Retrieved 2826 rows of alt data


,date,spend,transactions
0,2018-01-01,1667595.71,182683.82
1,2018-01-02,1880095.77,229058.71
2,2018-01-03,1869813.38,233150.78
3,2018-01-04,1775453.97,223530.40
4,2018-01-05,2102764.77,258916.25


In [13]:
# Compute rolling averages
df_alt['rolling_spend_7d'] = df_alt['spend'].rolling(window=7).mean()
df_alt['rolling_spend_30d'] = df_alt['spend'].rolling(window=30).mean()

# Create the visualization
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot Alt Data (Spend)
fig.add_trace(
    go.Scatter(x=df_alt['date'], y=df_alt['rolling_spend_30d'], name="Consumer Edge Spend (30d Avg)", line=dict(color='blue')),
    secondary_y=False
)

# Plot Stock Price
fig.add_trace(
    go.Scatter(x=df_market['date'], y=df_market['close'], name=f"{TICKER} Stock Price", line=dict(color='green')),
    secondary_y=True
)

fig.update_layout(
    title=f"{BRAND_NAME} - Alt Data vs Market Performance",
    xaxis_title="Date",
    height=600
)
fig.update_yaxes(title_text="Consumer Spend ($)", secondary_y=False)
fig.update_yaxes(title_text="Stock Price ($)", secondary_y=True)

fig.show()
